In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel('External Validation Predictions.xlsx')

In [ ]:
df.head()

,Residue_Index,UNIPROTID,Amino_Acid,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,...,Leucine_Propensity,RelSASA,Secondary_Enum,Cluster,Calculated Sum,Threshold1 >=0.1116,Regions detected for threshold1,Unnamed: 19,Threshold2 >=0.27763,Regions detected for threshold2
0,1,P|P03427|PB2,11,C,J,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.635686,0.942029,1,0.304074,1,d,NaN,1,NaN
1,2,P|P03427|PB2,4,H,J,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.689330,0.942029,0,0.167956,1,d,NaN,0,NaN
2,3,P|P03427|PB2,15,H,H,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.470975,0.130700,1,0.333016,1,d,NaN,1,NaN
3,4,P|P03427|PB2,8,H,H,0.142857,0.0,0.0,0.0,0.0,...,-0.142857,0.265970,0.130700,0,0.170216,1,d,NaN,0,NaN
4,5,P|P03427|PB2,9,H,H,0.285714,0.0,0.0,0.0,0.0,...,-0.142857,0.554636,0.130700,0,0.231534,1,d,NaN,0,NaN


In [ ]:
def extract_and_save_embeddings(protein_name, sequence, output_file="external_training_data_protein_embeddings.csv"):
    sequence = re.sub(r"[UZOB]", "X", sequence)  # Replace rare amino acids
    sequence = " ".join(sequence)  # Ensure space-separated format

    encoded_input = tokenizer(sequence, return_tensors='pt', add_special_tokens=True)

    with torch.no_grad():
        output = model(**encoded_input)

    # Extract last hidden states
    embeddings = output.last_hidden_state.squeeze(0)  # Shape: (L, 1024)

    # Remove CLS and SEP tokens (First & Last)
    embeddings = embeddings[1:-1]  # Shape: (Sequence Length, 1024)

    # Create DataFrame with embeddings
    residues = list(sequence.replace(" ", ""))  # Remove spaces for correct residue mapping
    df = pd.DataFrame(embeddings.numpy(), columns=[f"dim_{i}" for i in range(1024)])
    df.insert(0, "Residue", residues)  # Add residue column
    df.insert(0, "Protein", protein_name)  # Add protein name column

    # Save to CSV (append if file exists)
    df.to_csv(output_file, index=False, mode='a', header=not pd.io.common.file_exists(output_file))
    print(f"Embeddings for {protein_name} saved to {output_file}")

    return embeddings